# Setup

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from regex_engine import generator
import logging
import boto3
from botocore.exceptions import ClientError
import os
#import pyarrow
#pd.set_option("mode.dtype_backend", "pyarrow")

In [2]:
pd.__version__

'2.0.0rc0'

In [3]:
df = pd.read_csv("127JourneyDataExtract12Sep2018-18Sep2018.csv")

In [4]:
df.shape

(248048, 9)

In [5]:
df.head(5)

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,80236130,960,1194,12/09/2018 09:22,215,"Moorfields, Moorgate",12/09/2018 09:06,819,"Belvedere Road 2, South Bank"
1,80234557,900,4550,12/09/2018 08:59,564,"Somerset House, Strand",12/09/2018 08:44,183,"Riverlight North, Nine Elms"
2,80468055,1080,6428,18/09/2018 16:23,456,"Parkway, Camden Town",18/09/2018 16:05,121,"Baker Street, Marylebone"
3,80314115,1920,7158,14/09/2018 11:02,529,"Manresa Road, Chelsea",14/09/2018 10:30,121,"Baker Street, Marylebone"
4,80462483,1320,3627,18/09/2018 13:02,172,"Sumner Place, South Kensington",18/09/2018 12:40,121,"Baker Street, Marylebone"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248048 entries, 0 to 248047
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Rental Id          248048 non-null  int64 
 1   Duration           248048 non-null  int64 
 2   Bike Id            248048 non-null  int64 
 3   End Date           248048 non-null  object
 4   EndStation Id      248048 non-null  int64 
 5   EndStation Name    248048 non-null  object
 6   Start Date         248048 non-null  object
 7   StartStation Id    248048 non-null  int64 
 8   StartStation Name  248048 non-null  object
dtypes: int64(5), object(4)
memory usage: 17.0+ MB


# Filter appropriate file names

In [46]:
#!aws s3 ls s3://cycling.data.tfl.gov.uk/usage-stats/ > usage_data_name.txt

In [2]:
s3_client = boto3.client('s3')
response = s3_client.list_objects_v2(Bucket='cycling.data.tfl.gov.uk', Prefix = 'usage-stat')

In [3]:
raw_urls = []
for obj in response['Contents']:
    raw_urls.append(obj['Key'])

In [ ]:
raw_urls = [u.replace('usage-stats/', '').replace(' ', '') for u in raw_urls]
raw_urls = list(filter(None, raw_urls))
generate = generator()

regex = generate.numerical_range(14, 352)
p = re.compile(regex[:-1] + "JourneyDataExtract")
filtered_names_list = [f for f in raw_urls if (p.match(f) and f.endswith(".csv"))]
foo_names_list = filtered_names_list[:8]

In [ ]:
len(filtered_names_list) == (352 - 14 + 1)

False

In [85]:
BUCKET_NAME = 'cycling.data.tfl.gov.uk'
BUCKET_PREFIX = 'usage-stats'
DATA_DIR_PREFIX = 'data'

In [86]:
for f in foo_names_list:
    s3_client.download_file(BUCKET_NAME, f"{BUCKET_PREFIX}/{f}", f"{DATA_DIR_PREFIX}/{f}")

In [68]:
BASE_URL = "https://cycling.data.tfl.gov.uk/usage-stats/"

In [75]:
filtered_urls = [BASE_URL + f for f in filtered_names_list]

In [76]:
pd.Series(filtered_urls).to_csv("filtered_urls.txt", index = False, header = False)

# Download files from the website

In [ ]:
#!./download_data.sh foo_urls.txt 8 data/

# Upload data to S3

In [27]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [29]:
upload_file("127JourneyDataExtract12Sep2018-18Sep2018.csv", "tfl-cycling","foo/127JourneyDataExtract12Sep2018-18Sep2018.csv")

True